In [1]:
import h2o
import pandas as pd
h2o.init()
from h2o.estimators.word2vec import H2OWord2vecEstimator
from nltk.corpus import stopwords

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,2 days 17 hours 13 mins
H2O cluster timezone:,Europe/Vienna
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.5
H2O cluster version age:,26 days
H2O cluster name:,H2O_from_python_mackenzie_79hb59
H2O cluster total nodes:,1
H2O cluster free memory:,1.751 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


In [2]:
data_path = "/home/mackenzie/workspace/PycharmProjects/DAADRISE_AbusiveLangProject/featureExtraction/EnglishCleanedTrainingData (1).csv"
data = h2o.import_file(data_path)
#print(data)
STOP_WORDS = set(stopwords.words('english'))
new_stopwords = ['rt'] 
STOP_WORDS = STOP_WORDS.union(new_stopwords)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [3]:
def tokenize(sentences, stop_word = STOP_WORDS):
    tokenized = sentences.tokenize("\\W+")
    tokenized_lower = tokenized.tolower()
    tokenized_filtered = tokenized_lower[(tokenized_lower.nchar() >= 2) | (tokenized_lower.isna()),:]
    tokenized_words = tokenized_filtered[tokenized_filtered.grep("[0-9]",invert=True,output_logical=True),:]
    tokenized_words = tokenized_words[(tokenized_words.isna()) | (~ tokenized_words.isin(STOP_WORDS)),:]
    return tokenized_words

In [4]:
def predict(job_title,w2v, gbm):
    words = tokenize(h2o.H2OFrame(job_title).ascharacter())
    job_title_vec = w2v.transform(words, aggregate_method="AVERAGE")
    print(gbm.predict(test_data=job_title_vec))

In [5]:
print("Break tweets into sequence of words")
words = tokenize(data["cleaned_tweet"])

Break tweets into sequence of words


In [6]:
print("Build word2vec model")
w2v_model = H2OWord2vecEstimator(sent_sample_rate = 0.0, epochs = 10)
w2v_model.train(training_frame=words)

Build word2vec model
word2vec Model Build progress: |██████████████████████████████████████████| 100%


In [7]:
print("Calculate a vector for each tweet")
tweet_vecs = w2v_model.transform(words, aggregate_method = "AVERAGE")
print(tweet_vecs)

Calculate a vector for each tweet


C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,C40,C41,C42,C43,C44,C45,C46,C47,C48,C49,C50,C51,C52,C53,C54,C55,C56,C57,C58,C59,C60,C61,C62,C63,C64,C65,C66,C67,C68,C69,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79,C80,C81,C82,C83,C84,C85,C86,C87,C88,C89,C90,C91,C92,C93,C94,C95,C96,C97,C98,C99,C100
0.212917,0.230906,-0.0886541,0.294202,0.123474,0.283383,0.0074332,-0.170523,-0.111196,0.455966,-0.0219766,-0.196902,-0.152014,-0.0511684,-0.181307,0.0530709,-0.102852,-0.0634184,0.105651,-0.0472447,-0.0850808,0.328263,-0.0339493,-0.14443,0.0367176,-0.16038,0.146651,-0.14258,-0.0286465,-0.310238,0.139351,-0.448082,0.157095,0.0649458,-0.00299131,-0.452157,0.163194,-0.259241,-0.0895197,0.167221,0.102373,0.103336,-0.0146127,0.136443,-0.226902,0.0412764,0.243821,-0.0168933,-0.34183,0.100697,-0.0851984,0.305818,-0.126877,-0.0414445,0.155664,0.0923565,-0.323685,0.0547481,-0.222182,-0.00866119,-0.0252524,0.106256,0.21133,0.291336,-0.279539,0.167368,-0.00947552,0.195283,0.0395001,0.391614,0.164911,-0.0164005,0.00528101,0.0982212,0.127626,0.23609,-0.307877,0.134955,-0.321039,-0.226808,-0.145472,0.237303,0.209097,0.202548,-0.173082,-0.0214403,0.160446,-0.0860858,0.0604127,-0.0473036,-0.231582,0.151163,-0.0824016,0.165118,0.025137,0.0274065,-0.0830896,-0.036089,0.197341,-0.0444415
0.272519,0.0706675,0.0403761,0.204777,0.109998,0.00271851,-0.113222,-0.147757,-0.0178177,0.368064,-0.0122278,0.0837669,0.0113708,0.0318,-0.0378209,-0.0376233,-0.104951,0.119099,0.0861896,-0.0473067,-0.040789,0.0861963,0.00367173,0.0775075,0.183846,0.162582,-0.0530834,-0.0906191,-0.0130155,0.0635604,0.161258,-0.174427,-0.0627454,0.0399699,-0.0162873,-0.174503,0.148872,-0.0998424,-0.00349117,-0.0126977,0.037833,0.175795,-0.0488127,0.16807,0.015645,-0.0859147,0.0361021,0.138341,0.0265329,0.0182833,-0.229636,0.150919,-0.109991,-0.0662375,0.15793,0.00062945,-0.0664053,0.0331392,-0.156291,0.0275471,-0.0776486,-0.00859467,-0.0621013,0.105882,-0.205898,-0.0959654,-0.166477,0.145708,0.042405,0.267982,0.0812777,0.00345463,-0.107096,0.0368292,0.00576519,0.165786,-0.0503758,0.0957577,-0.168819,-0.00484079,0.00577457,-0.106171,-0.0158881,0.0800753,-0.0935556,-0.00795182,0.1375,-0.00862966,-0.0341317,-0.0676855,-0.0402984,0.0260642,-0.102053,0.122878,0.0553434,0.181715,0.0628671,-0.109709,0.156089,-0.0107078
0.137377,0.135227,-0.155909,0.306672,0.201073,0.135417,0.0260809,-0.279848,-0.160381,0.248433,0.174452,-0.0602271,-0.119057,-0.0868848,-0.0357339,0.0747303,-0.0952025,0.000290709,0.182728,0.00143895,-0.0388331,0.0279358,0.0195431,0.103572,0.182855,-0.0514792,0.00926028,-0.196638,0.00384504,-0.0482662,0.00851427,-0.129896,0.136706,0.0644539,0.0181932,-0.220775,0.13669,-0.0661809,0.0144632,-0.0544888,0.160445,0.0938703,-0.162605,0.0265653,-0.10652,-0.0485849,-0.000426453,-0.0874922,0.0667362,0.00919219,0.100416,0.091116,-0.267946,-0.0351616,0.00355651,-0.158502,-0.208762,-0.0267317,-0.0786899,0.156366,-0.0780966,0.287318,0.023403,0.0830671,-0.115133,0.0146158,-0.182134,0.0171192,0.12864,0.116676,0.245326,0.0267274,-0.0762772,-0.114487,0.0530546,0.342635,-0.136834,-0.0138162,-0.188383,-0.0217207,-0.0010352,0.0535919,0.0586338,-0.0118348,-0.210758,0.0219197,0.143401,-0.085031,-0.152885,-0.103069,-0.00723587,0.0081848,-0.189029,-0.0573201,0.0430598,0.0255829,0.00951744,-0.0593675,0.294677,-0.0985883
0.269022,0.0442898,0.0551614,0.106022,0.0502941,0.0273017,-0.0762859,-0.298209,-0.0645253,0.181468,-0.0734535,-0.0240435,0.0479189,-0.0358202,0.0710023,-0.103601,-0.124787,0.127674,-0.0725298,-0.0269772,-0.0443976,-0.0369043,0.0150055,0.140467,0.190375,0.0519342,-0.0260513,-0.0722796,-0.0736632,0.0165435,0.10041,-0.127752,-0.143536,0.0180546,0.0421104,-0.201786,0.180373,0.083104,-0.0122148,-0.0407694,0.00833777,0.186273,-0.0228889,0.00464309,-0.0807132,0.0404103,-0.0116234,-0.0547478,0.0317588,0.13344,-0.00957205,0.0443209,-0.0523939,-0.00942753,0.102306,-0.140174,0.0665564,0.0719263,-0.0830416,-0

In [8]:
tweets_pd = tweet_vecs.as_data_frame()
print(tweets_pd.isna())

          C1     C2     C3     C4     C5     C6     C7     C8     C9    C10  \
0      False  False  False  False  False  False  False  False  False  False   
1      False  False  False  False  False  False  False  False  False  False   
2      False  False  False  False  False  False  False  False  False  False   
3      False  False  False  False  False  False  False  False  False  False   
4      False  False  False  False  False  False  False  False  False  False   
5      False  False  False  False  False  False  False  False  False  False   
6      False  False  False  False  False  False  False  False  False  False   
7      False  False  False  False  False  False  False  False  False  False   
8      False  False  False  False  False  False  False  False  False  False   
9      False  False  False  False  False  False  False  False  False  False   
10     False  False  False  False  False  False  False  False  False  False   
11     False  False  False  False  False  False  Fal

[19808 rows x 100 columns]


In [9]:
print(tweets_pd[tweets_pd.isna().any(axis=1)][tweets_pd.columns].head())

      C1  C2  C3  C4  C5  C6  C7  C8  C9  C10  ...  C91  C92  C93  C94  C95  \
1922 NaN NaN NaN NaN NaN NaN NaN NaN NaN  NaN  ...  NaN  NaN  NaN  NaN  NaN   
3701 NaN NaN NaN NaN NaN NaN NaN NaN NaN  NaN  ...  NaN  NaN  NaN  NaN  NaN   
4166 NaN NaN NaN NaN NaN NaN NaN NaN NaN  NaN  ...  NaN  NaN  NaN  NaN  NaN   
4550 NaN NaN NaN NaN NaN NaN NaN NaN NaN  NaN  ...  NaN  NaN  NaN  NaN  NaN   
5570 NaN NaN NaN NaN NaN NaN NaN NaN NaN  NaN  ...  NaN  NaN  NaN  NaN  NaN   

      C96  C97  C98  C99  C100  
1922  NaN  NaN  NaN  NaN   NaN  
3701  NaN  NaN  NaN  NaN   NaN  
4166  NaN  NaN  NaN  NaN   NaN  
4550  NaN  NaN  NaN  NaN   NaN  
5570  NaN  NaN  NaN  NaN   NaN  

[5 rows x 100 columns]


In [10]:
h2o.download_all_logs()

Writing H2O logs to ./h2ologs_20190715_100011.zip


'./h2ologs_20190715_100011.zip'